In [1]:
!nvidia-smi

Thu Oct 31 09:39:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.18                 Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A3000 Laptop GPU  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   61C    P8              15W /  80W |    555MiB /  6144MiB |     44%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [16]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.6252, 0.2640, 0.4693],
        [0.5900, 0.9599, 0.9348]], device='cuda:0')

In [14]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [17]:
Y + Z

tensor([[1.6252, 1.2640, 1.4693],
        [1.5900, 1.9599, 1.9348]], device='cuda:0')

In [19]:
Z.cuda(0) is Z

True

In [20]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [21]:
net(X)

tensor([[-0.2773],
        [-0.2773]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
net[0].weight.data.device

device(type='cuda', index=0)